<a href="https://colab.research.google.com/github/rimgosu/ColabBackup/blob/master/ex012_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# input = 가는
# output = 말이 고와야 오는 말이 곱다
# 첫 마디 입력시 뒤에 나올 말 생성

In [3]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""
# 바로 전의 말만 확인하고 텍스트를 생성하는 게 아니라 전에 나왔던 모든 말을 확인하고
# 텍스트를 생성하기 때문에 "말이" 다음에 나오는 말들을 생성 가능

In [5]:
# 띄어쓰기를 기준으로 단어 분리
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts([text])

t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [6]:
# 훈련 데이터 생성
sequences = []
for line in text.split('\n'):
  # 단어를 정수로 변경
  encoded = t.texts_to_sequences([line])[0]

  # 2개 이상의 조합 생성
  for i in range(1,len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

In [7]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 6

sequences = pad_sequences(sequences, maxlen = max_len)
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [10]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1] # 문제데이터
y = sequences[:,-1] # 정답데이터

In [11]:
# 원핫인코딩
from tensorflow.keras.utils import to_categorical

y_en = to_categorical(y,num_classes = 12)
y_en

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [12]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras import Sequential

model1 = Sequential()

model1.add(Embedding(12,10,input_length = 5))
model1.add(SimpleRNN(32))
model1.add(Dense(units = 12, activation = 'softmax'))

model1.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model1.fit(X, y_en, epochs = 200)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 2.4904 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 2.4778 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 2.4653 - accuracy: 0.0909
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 2.4527 - accuracy: 0.1818
Epoch 5/200
1/1 [==============================] - 0s 10ms/step - loss: 2.4400 - accuracy: 0.0909
Epoch 6/200
1/1 [==============================] - 0s 10ms/step - loss: 2.4271 - accuracy: 0.3636
Epoch 7/200
1/1 [==============================] - 0s 13ms/step - loss: 2.4138 - accuracy: 0.3636
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 2.4001 - accuracy: 0.4545
Epoch 9/200
1/1 [==============================] - 0s 10ms/step - loss: 2.3859 - accuracy: 0.4545
Epoch 10/200
1/1 [==============================] - 0s 12ms/step - loss: 2.3710 - accuracy: 0.4545
Epoch 11/200


In [13]:
# 학습 모델 사용해서 텍스트 생성기능 만들기
# 사용할 모델, 토크나이저, 입력 단어, 생성할 단어 수
def make_sentence(model, t, current_word, n):
  init_word = current_word # 마지막에 같이 출력하기 위함
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    print(encoded) # 번호 확인
    print(current_word) # 입력 단어 확인
    # 데이터를 같은 길이(5)로 맞춰주기
    encoded = pad_sequences([encoded], maxlen = max_len - 1)
    # 입력한 단어로 예측하기
    result = model.predict(encoded, verbose = 0).argmax(axis= -1)
    for word, index in t.word_index.items():
      if index == result:
        break

    # 현재단어 + " " + 예측단어를 현재 단어로 변경
    current_word = current_word + " " + word

    sentence = sentence + " " + word
  # 문장 생성
  sentence = init_word + sentence

  return sentence

In [15]:
# 인공지능 딥러닝 모델, 토큰화 모델, 시작 단어
# 경마장에 있는 말이 뛰고 있다
# 그의 말이 법이다
# 가는 말이 고와야 오는 말이 곱다
print(make_sentence(model1, t, '경마장에', 4))

[2]
경마장에
[2, 3]
경마장에 있는
[2, 3, 1]
경마장에 있는 말이
[2, 3, 1, 4]
경마장에 있는 말이 뛰고
경마장에 있는 말이 뛰고 있다


In [18]:
print(make_sentence(model1, t, '그의', 2))

[6]
그의
[6, 1]
그의 말이
그의 말이 법이다


In [19]:
print(make_sentence(model1, t, '가는', 5))

[8]
가는
[8, 1]
가는 말이
[8, 1, 9]
가는 말이 고와야
[8, 1, 9, 10]
가는 말이 고와야 오는
[8, 1, 9, 10, 1]
가는 말이 고와야 오는 말이
가는 말이 고와야 오는 말이 곱다
